In [1]:
from datetime import datetime
from scipy import stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.ticker as ticker
import os
import statistics

%matplotlib inline

In [2]:
contract_loc = 'data/sc_vol_adj_futures/'
contract_files = list(file for file in os.listdir(contract_loc) if file[-3:] == 'csv')

In [3]:
"""
Create a dictionary where the key is the ticker
and the value is a pandas dataframe of the OHLC time series
Run this every weekday at 13:00 GMT+7
"""
data_futures = {}
for file in contract_files:
    data_futures['-'.join((file.split('.')[0].split('-')[0][:-3], file.split('.')[0].split('-')[-1]))] = pd.read_csv(f'{contract_loc}/{file}', usecols=[0, 1], index_col=0, parse_dates=True)

In [4]:
def volatility(ts, lookback=24):
    """
    Input:  Price time series, Look back period
    Output: Standard deviation of the percent change
    """
    return ts.pct_change().rolling(lookback).std().iloc[-1]

def ewm_volatility(ts, lookback=36):
    """
    Input:  Price time series, Look back period
    Output: Standard deviation of the percent change using ewm calculation
    """
    return ts.pct_change().ewm(span=lookback).std().iloc[-1]

In [5]:
# Create an empty DataFrame to store score
ins_risk_table = pd.DataFrame(columns=['ticker', 'last_date', 'ins_risk_starter', 'ins_risk_staunch'])

# How many (series) candles back for std dev calculation?
vola_window = 24

# Loop the dictionary and calculate the momentum_score, then append it to pandas
for ticker, timeseries in data_futures.items():
    ins_risk_starter = volatility(timeseries[' Last']) * 16
    ins_risk_staunch = ewm_volatility(timeseries[' Last']) * 16
    last_date = timeseries.index[-1]
    ins_risk_table = ins_risk_table.append({'ticker': ticker,
                                            'last_date': last_date,
                                            'ins_risk_starter': ins_risk_starter,
                                            'ins_risk_staunch': ins_risk_staunch},
                                           ignore_index=True)

In [6]:
ins_risk_table.to_clipboard()
ins_risk_table

,ticker,last_date,ins_risk_starter,ins_risk_staunch
0,F2MX-EUREX,2020-12-10,0.096437,0.131811
1,OJ-ICEUS,2020-12-10,0.259914,0.295033
2,FESX-EUREX,2020-12-10,0.226623,0.196528
3,FGBS-EUREX,2020-12-10,0.004466,0.003911
4,RC-ICEEU,2020-12-10,0.224294,0.213646
...,...,...,...,...
75,AP-SFE,2020-09-15,0.189978,0.203053
76,GF-CME,2020-12-10,0.201155,0.180119
77,6J-CME,2020-12-10,0.082379,0.063371
78,FTDX-EUREX,2020-12-10,0.162184,0.181207
